In [5]:
import face_recognition
import pickle
import os
import cv2

FRAME_THICKNESS = 2
FONT_THICKNESS = 1
TOLERANCE = 0.4

In [ ]:
LABELED_FACES_DIR = "../Dataset/Training/"
TEST_DIR = "../Dataset/Testing/"

current_faces = []
known_faces = []
known_names = []

#CREATE ENCODING FOR EACH IMAGE
for name in os.listdir(LABELED_FACES_DIR):
    for filename in os.listdir(f"{LABELED_FACES_DIR}/{name}"):
        image = face_recognition.load_image_file(f"{LABELED_FACES_DIR}/{name}/{filename}")
        encoding = face_recognition.face_encodings(image)[0]
        known_faces.append(encoding)
        known_names.append(name)

#EXPORT ENCODINGS AND LABELS INTO A .DAT FILE
with open('known_faces.dat','wb') as f:
    pickle.dump(known_faces, f)

with open('known_names.dat','wb') as f:
    pickle.dump(known_names, f)

In [3]:
#IMPORT ENCODINGS AND LABELS OF KNOWN FACES

with open('known_faces.dat','rb') as f:
    known_faces = pickle.load(f)

with open('known_names.dat','rb') as f:
    known_names = pickle.load(f)

In [ ]:
UNLABELED_FACES_DIR = "../Dataset/Validation"


#LOOK AT UNKNOWN IMAGES AND CLASSIFY THEM
for filename in os.listdir(UNLABELED_FACES_DIR):
    print(filename)
    image = face_recognition.load_image_file(f"{UNLABELED_FACES_DIR}/{filename}")
    locations = face_recognition.face_locations(image, model='cnn')
    encodings = face_recognition.face_encodings(image, locations)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    for face_encoding, face_location in zip(encodings, locations):
        results = face_recognition.compare_faces(known_faces, face_encoding, TOLERANCE)
        match = None
        if True in results:
            match = known_names[results.index(True)]
            print(f"Match found: {match}")

            #A list of tuples of found face locations in css (top, right, bottom, left) order
            top_left = (face_location[3]-10, face_location[0]-50)
            bottom_right = (face_location[1]+10, face_location[2]+50)
            color = [0, 255, 0]
            cv2.rectangle(image, top_left, bottom_right, color, FRAME_THICKNESS)

            top_left = (face_location[3] - 11, face_location[2] + 50)
            bottom_right = (face_location[1] + 11, face_location[2] + 72)
            color = [0, 255, 0]
            cv2.rectangle(image, top_left, bottom_right, color, cv2.FILLED)

            cv2.putText(image, match, (face_location[3], face_location[2]+65), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0,0,0), FONT_THICKNESS)

    cv2.imshow(filename, image)
    cv2.waitKey(1000)

In [6]:
input_video = cv2.VideoCapture('../Dataset/Video1.mp4')
length = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter('output.avi', fourcc, 29.97, (640,360))

face_locations = []
face_encodings = []
frame_number = 1400

while True:
    # Grab a single frame of video
    ret, frame = input_video.read()
    frame_number += 1

    # Quit when the input video file ends
    if not ret:
        break
    if frame_number == 1500:
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    locations = face_recognition.face_locations(rgb_frame, model='cnn')
    encodings = face_recognition.face_encodings(rgb_frame, locations)

    for face_encoding, face_location in zip(encodings, locations):
        results = face_recognition.compare_faces(known_faces, face_encoding, TOLERANCE)
        match = None
        if True in results:
            match = known_names[results.index(True)]

            #A list of tuples of found face locations in css (top, right, bottom, left) order
            top_left = (face_location[3]-10, face_location[0]-50)
            bottom_right = (face_location[1]+10, face_location[2]+50)
            color = [0, 255, 0]
            cv2.rectangle(frame, top_left, bottom_right, color, FRAME_THICKNESS)

            top_left = (face_location[3] - 11, face_location[2] + 50)
            bottom_right = (face_location[1] + 11, face_location[2] + 72)
            color = [0, 255, 0]
            cv2.rectangle(frame, top_left, bottom_right, color, cv2.FILLED)

            cv2.putText(frame, match, (face_location[3], face_location[2]+65), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0,0,0), FONT_THICKNESS)

    # Write the resulting image to the output video file
    print("Writing frame {} / {}".format(frame_number, length))
    output_video.write(frame)

# All done!
input_video.release()
cv2.destroyAllWindows()


Writing frame 1401 / 2391
Writing frame 1402 / 2391
Writing frame 1403 / 2391
Writing frame 1404 / 2391
Writing frame 1405 / 2391
Writing frame 1406 / 2391
Writing frame 1407 / 2391
Writing frame 1408 / 2391
Writing frame 1409 / 2391
Writing frame 1410 / 2391
Writing frame 1411 / 2391
Writing frame 1412 / 2391
Writing frame 1413 / 2391
Writing frame 1414 / 2391
Writing frame 1415 / 2391
Writing frame 1416 / 2391
Writing frame 1417 / 2391
Writing frame 1418 / 2391
Writing frame 1419 / 2391
Writing frame 1420 / 2391
Writing frame 1421 / 2391
Writing frame 1422 / 2391
Writing frame 1423 / 2391
Writing frame 1424 / 2391
Writing frame 1425 / 2391
Writing frame 1426 / 2391
Writing frame 1427 / 2391
Writing frame 1428 / 2391
Writing frame 1429 / 2391
Writing frame 1430 / 2391
Writing frame 1431 / 2391
Writing frame 1432 / 2391
Writing frame 1433 / 2391
Writing frame 1434 / 2391
Writing frame 1435 / 2391
Writing frame 1436 / 2391
Writing frame 1437 / 2391
Writing frame 1438 / 2391
Writing fram

In [ ]:
# import face_recognition
# import cv2

# # This is a demo of running face recognition on a video file and saving the results to a new video file.
# #
# # PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# # OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# # specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# # Open the input movie file
# input_movie = cv2.VideoCapture("hamilton_clip.mp4")
# length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))

# # Create an output movie file (make sure resolution/frame rate matches input video!)
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# output_movie = cv2.VideoWriter('output.avi', fourcc, 29.97, (640, 360))

# # Load some sample pictures and learn how to recognize them.
# lmm_image = face_recognition.load_image_file("lin-manuel-miranda.png")
# lmm_face_encoding = face_recognition.face_encodings(lmm_image)[0]

# al_image = face_recognition.load_image_file("alex-lacamoire.png")
# al_face_encoding = face_recognition.face_encodings(al_image)[0]

# known_faces = [
#     lmm_face_encoding,
#     al_face_encoding
# ]

# # Initialize some variables
# face_locations = []
# face_encodings = []
# face_names = []
# frame_number = 0

# while True:
#     # Grab a single frame of video
#     ret, frame = input_movie.read()
#     frame_number += 1

#     # Quit when the input video file ends
#     if not ret:
#         break

#     # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
#     rgb_frame = frame[:, :, ::-1]

#     # Find all the faces and face encodings in the current frame of video
#     face_locations = face_recognition.face_locations(rgb_frame)
#     face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

#     face_names = []
#     for face_encoding in face_encodings:
#         # See if the face is a match for the known face(s)
#         match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.50)

#         # If you had more than 2 faces, you could make this logic a lot prettier
#         # but I kept it simple for the demo
#         name = None
#         if match[0]:
#             name = "Lin-Manuel Miranda"
#         elif match[1]:
#             name = "Alex Lacamoire"

#         face_names.append(name)

#     # Label the results
#     for (top, right, bottom, left), name in zip(face_locations, face_names):
#         if not name:
#             continue

#         # Draw a box around the face
#         cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

#         # Draw a label with a name below the face
#         cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
#         font = cv2.FONT_HERSHEY_DUPLEX
#         cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

#     # Write the resulting image to the output video file
#     print("Writing frame {} / {}".format(frame_number, length))
#     output_movie.write(frame)

# # All done!
# input_movie.release()
# cv2.destroyAllWindows()